In [ ]:
include("./src/MPO_common.jl")
using .MPO_common
using JLD2
using BenchmarkTools
using ITensors
using KrylovKit
using LinearAlgebra
include("./src/iDMRG.jl")
using .iDMRG

In [2]:
J1 = 4 * 1.0
Delta = 4 * 0.5
J2 = 4 * 0.2
Ly = 6
uc = Ly

oplist = MPOsum()
for i = 1:Ly
    mpoadd!(oplist, J1 + Delta, (i, "Sz"), (mod1(i + 1, 6), "Sz"))
    mpoadd!(oplist, J1, (i, "Sx"), (mod1(i + 1, 6), "Sx"))
    mpoadd!(oplist, J1, (i, "Sy"), (mod1(i + 1, 6), "Sy"))
    mpoadd!(oplist, J1 + Delta, (i, "Sz"), (mod1(i + 1, 6) + 6, "Sz"))
    mpoadd!(oplist, J1, (i, "Sx"), (mod1(i + 1, 6) + 6, "Sx"))
    mpoadd!(oplist, J1, (i, "Sy"), (mod1(i + 1, 6) + 6, "Sy"))
    mpoadd!(oplist, J1 + Delta, (i, "Sz"), (i + 6, "Sz"))
    mpoadd!(oplist, J1, (i, "Sx"), (i + 6, "Sx"))
    mpoadd!(oplist, J1, (i, "Sy"), (i + 6, "Sy"))

    mpoadd!(oplist, J2, (i, "Sz"), (mod1(i + 2, 6) + 6, "Sz"))
    mpoadd!(oplist, J2, (i, "Sx"), (mod1(i + 2, 6) + 6, "Sx"))
    mpoadd!(oplist, J2, (i, "Sy"), (mod1(i + 2, 6) + 6, "Sy"))
    mpoadd!(oplist, J2, (i, "Sz"), (mod1(i + 1, 6) + 12, "Sz"))
    mpoadd!(oplist, J2, (i, "Sx"), (mod1(i + 1, 6) + 12, "Sx"))
    mpoadd!(oplist, J2, (i, "Sy"), (mod1(i + 1, 6) + 12, "Sy"))
    mpoadd!(oplist, J2, (i, "Sz"), (mod1(i - 1, 6) + 6, "Sz"))
    mpoadd!(oplist, J2, (i, "Sx"), (mod1(i - 1, 6) + 6, "Sx"))
    mpoadd!(oplist, J2, (i, "Sy"), (mod1(i - 1, 6) + 6, "Sy"))
end

sites = siteinds("S=1/2", uc)
iH = op_to_hm_inf(oplist, sites);

leftinds = Vector{Index}(undef, uc)
for i = 1:uc
    leftinds[i] = commonind(iH[mod1(i - 1, uc)], iH[i])
end

resultvalue = iDMRG_init(sites, leftinds)
psis0 = resultvalue[1]
LP10 = resultvalue[2]
RPuc = resultvalue[3]
L_2 = resultvalue[4]
L_4 = resultvalue[5]
println("finish initialization")
flush(stdout)
#End of initialization

psis0, lp10, rpuc, l2, l4 = iDMRG_init(sites, leftinds)
maxdimlist = [50, 100, 250, 250, 200, 200]
sweeplist = [2, 2, 3, 10, 10, 20];
updatedlist = idmrg_general(iH, psis0, lp10, rpuc, l2, l4; maxdim=maxdimlist[1], cutoff=1e-2, nsweeps=1)

for l = 1:3
    println("This is global sweep $l")
    if l <= 3
        dmrgsweeptime = @elapsed updatedlist = idmrg_general(iH, updatedlist[1], updatedlist[2], updatedlist[3], updatedlist[4], updatedlist[5]; nsweeps=1, krylovdimmax=3, niter=1, maxdim=maxdimlist[l], cutoff=1e-10,noise=1e-2)
    end
    for i = 1:sweeplist[l] #We turn on the noise in the first few runs
        println("Sweep $i")
        flush(stdout)
        dmrgsweeptime = @elapsed updatedlist = idmrg_general(iH, updatedlist[1], updatedlist[2], updatedlist[3], updatedlist[4], updatedlist[5]; nsweeps=1, krylovdimmax=2, niter=1, maxdim=maxdimlist[l], cutoff=1e-10)
        flush(stdout)
    end
end


finish initialization
Maximum bond dimension is: 2
Energy is: -2.9073261161463777
Lambda overlap is: 0.8372434609252518
This is global sweep 1
Maximum bond dimension is: 16
Energy is: -1.7987064655521128
Lambda overlap is: 0.9814149324119115
Sweep 1
Maximum bond dimension is: 50
Energy is: -1.7588015496597424
Lambda overlap is: 0.9762713972316766
Sweep 2
Maximum bond dimension is: 50
Energy is: -2.2335139052013964
Lambda overlap is: 0.9900673454133663
This is global sweep 2
Maximum bond dimension is: 100
Energy is: -2.357524605580558
Lambda overlap is: 0.9929384990388126
Sweep 1
Maximum bond dimension is: 100
Energy is: -2.3584101238681114
Lambda overlap is: 0.999433555400534
Sweep 2
Maximum bond dimension is: 100
Energy is: -2.365936329366164
Lambda overlap is: 0.9998245192672424
This is global sweep 3
Maximum bond dimension is: 250
Energy is: -2.3787911478941957
Lambda overlap is: 0.9989845903496786
Sweep 1
Maximum bond dimension is: 250
Energy is: -2.3771015086035825
Lambda overlap 

In [6]:
Base.summarysize(updatedlist)/1024^3

0.08196596801280975